<a href="https://colab.research.google.com/github/jainakshita16/Unlocking-Customer-Insights-with-PySpark---Retail-Sales-Analysis/blob/main/Unlocking%20Customer%20Insights%20with%20PySpark%20-%20Retail%20Sales%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving part-00000 (1).txt to part-00000 (1).txt


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving part-00000 (2).txt to part-00000 (2).txt


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving part-00000 (3).txt to part-00000 (3).txt


In [ ]:
import os
os.listdir()


['.config',
 'part-00000 (2).txt',
 'part-00000 (1).txt',
 'part-00000 (3).txt',
 'sample_data']

In [ ]:
os.rename("part-00000 (1).txt", "customers.csv")
os.rename("part-00000 (2).txt", "order_items.csv")
os.rename("part-00000 (3).txt", "orders.csv")


In [ ]:
os.listdir()


['.config', 'orders.csv', 'customers.csv', 'order_items.csv', 'sample_data']

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar -xzf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RetailSalesAnalysis").getOrCreate()


In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType

# Define schemas
customer_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("customer_fname", StringType(), True),
    StructField("customer_lname", StringType(), True),
    StructField("customer_email", StringType(), True),
    StructField("customer_password", StringType(), True),
    StructField("customer_street", StringType(), True),
    StructField("customer_city", StringType(), True),
    StructField("customer_state", StringType(), True),
    StructField("customer_zipcode", IntegerType(), True),
])

orders_schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("order_date", DateType(), True),
    StructField("order_customer_id", IntegerType(), True),
    StructField("order_status", StringType(), True),
])

order_items_schema = StructType([
    StructField("order_item_id", IntegerType(), True),
    StructField("order_item_order_id", IntegerType(), True),
    StructField("order_item_product_id", IntegerType(), True),
    StructField("order_item_quantity", IntegerType(), True),
    StructField("order_item_subtotal", FloatType(), True),
    StructField("order_item_product_price", FloatType(), True),
])

# Load CSV files in Colab
customer_df = spark.read.csv("customers.csv", header=False, schema=customer_schema)
orders_df = spark.read.csv("orders.csv", header=False, schema=orders_schema)
order_items_df = spark.read.csv("order_items.csv", header=False, schema=order_items_schema)


In [ ]:
customers_orders_df=customer_df.join(orders_df,customer_df['customer_id']==orders_df['order_customer_id'])

In [ ]:
customers_orders_df.show(1)

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+----------+-----------------+------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|order_id|order_date|order_customer_id|order_status|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+----------+-----------------+------------+
|      11599|          Mary|        Malone|     XXXXXXXXX|        XXXXXXXXX|8708 Indian Horse...|      Hickory|            NC|           28601|       1|2013-07-25|            11599|      CLOSED|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+--------+----------+-----------------+------------+
only showing top 1 row



In [ ]:
customers_orders_df.select('customer_id','order_id','order_date','order_status').orderBy('customer_id').show(10)

+-----------+--------+----------+---------------+
|customer_id|order_id|order_date|   order_status|
+-----------+--------+----------+---------------+
|          1|   22945|2013-12-13|       COMPLETE|
|          2|   33865|2014-02-18|       COMPLETE|
|          2|   15192|2013-10-29|PENDING_PAYMENT|
|          2|   57963|2013-08-02|        ON_HOLD|
|          2|   67863|2013-11-30|       COMPLETE|
|          3|   35158|2014-02-26|       COMPLETE|
|          3|   57617|2014-07-24|       COMPLETE|
|          3|   22646|2013-12-11|       COMPLETE|
|          3|   23662|2013-12-19|       COMPLETE|
|          3|   56178|2014-07-15|        PENDING|
+-----------+--------+----------+---------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import struct

In [ ]:
customers_orders_df.select('customer_id',struct('order_id','order_date','order_status').alias('order_details')).orderBy('customer_id').show(10)

+-----------+--------------------+
|customer_id|       order_details|
+-----------+--------------------+
|          1|{22945, 2013-12-1...|
|          2|{33865, 2014-02-1...|
|          2|{15192, 2013-10-2...|
|          2|{57963, 2013-08-0...|
|          2|{67863, 2013-11-3...|
|          3|{35158, 2014-02-2...|
|          3|{57617, 2014-07-2...|
|          3|{22646, 2013-12-1...|
|          3|{23662, 2013-12-1...|
|          3|{56178, 2014-07-1...|
+-----------+--------------------+
only showing top 10 rows



In [ ]:
customer_order_struct=customers_orders_df.select('customer_id',struct('order_id','order_date','order_status').alias('order_details'))

In [ ]:
from pyspark.sql.functions import collect_list

In [ ]:
final_df=customer_order_struct.groupBy('customer_id').agg(collect_list('order_details').alias('order_details')).orderBy('customer_id')

In [ ]:
final_df.show(2,truncate=False)

+-----------+----------------------------------------------------------------------------------------------------------------------------------+
|customer_id|order_details                                                                                                                     |
+-----------+----------------------------------------------------------------------------------------------------------------------------------+
|1          |[{22945, 2013-12-13, COMPLETE}]                                                                                                   |
|2          |[{15192, 2013-10-29, PENDING_PAYMENT}, {33865, 2014-02-18, COMPLETE}, {57963, 2013-08-02, ON_HOLD}, {67863, 2013-11-30, COMPLETE}]|
+-----------+----------------------------------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [ ]:
final_df.coalesce(1).write.mode("overwrite").json("/content/final_output_json")


In [ ]:
customer_details=customer_df. \
join(orders_df,customer_df['customer_id']==orders_df['order_customer_id']). \
join(order_items_df,orders_df['order_id']==order_items_df['order_item_order_id'])

In [ ]:
from pyspark.sql.functions import struct, collect_list

In [ ]:
denorm_df=customer_details. \
select('customer_id','customer_fname','customer_lname','customer_email','order_id','order_date','order_status',struct('order_item_id','order_item_product_id','order_item_subtotal').alias('order_item_details')). \
groupBy('customer_id','customer_fname','customer_lname','customer_email','order_id','order_date','order_status'). \
agg(collect_list('order_item_details').alias('order_item_details')). \
orderBy('customer_id'). \
select('customer_id','customer_fname','customer_lname','customer_email',struct('order_id','order_date','order_status','order_item_details').alias('order_details')). \
groupBy('customer_id','customer_fname','customer_lname','customer_email'). \
agg(collect_list('order_details').alias('order_details')). \
orderBy('customer_id')

In [ ]:
denorm_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_fname: string (nullable = true)
 |-- customer_lname: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- order_details: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- order_id: integer (nullable = true)
 |    |    |-- order_date: date (nullable = true)
 |    |    |-- order_status: string (nullable = true)
 |    |    |-- order_item_details: array (nullable = false)
 |    |    |    |-- element: struct (containsNull = false)
 |    |    |    |    |-- order_item_id: integer (nullable = true)
 |    |    |    |    |-- order_item_product_id: integer (nullable = true)
 |    |    |    |    |-- order_item_subtotal: float (nullable = true)



In [ ]:
denorm_df.coalesce(1).write.mode('overwrite').json('/content/denorm_output')


In [ ]:
json_df = spark.read.json('/content/denorm_output')


In [ ]:
from pyspark.sql.functions import explode,filter

In [ ]:
json_df.select('customer_id','customer_fname',explode('order_details').alias('order_details')). \
filter('order_details.order_date LIKE "2014-01-01%"'). \
orderBy('customer_id'). \
select('customer_id','customer_fname','order_details.order_id','order_details.order_date','order_details.order_status')

DataFrame[customer_id: bigint, customer_fname: string, order_id: bigint, order_date: string, order_status: string]

In [ ]:
from pyspark.sql.functions import date_format,col

In [ ]:
flatten=json_df.select('customer_id','customer_fname',explode('order_details').alias('order_details')). \
select('customer_id','customer_fname',col('order_details.order_date').alias('order_date'),col('order_details.order_id').alias('order_id'),col('order_details.order_status').alias('order_status'),explode('order_details.order_item_details').alias('order_item_details')). \
select('customer_id','customer_fname','order_date','order_id','order_status','order_item_details.order_item_id','order_item_details.order_item_product_id','order_item_details.order_item_subtotal')


In [ ]:
flatten.show(10)

+-----------+--------------+----------+--------+---------------+-------------+---------------------+-------------------+
|customer_id|customer_fname|order_date|order_id|   order_status|order_item_id|order_item_product_id|order_item_subtotal|
+-----------+--------------+----------+--------+---------------+-------------+---------------------+-------------------+
|          1|       Richard|2013-12-13|   22945|       COMPLETE|        57439|                  191|             499.95|
|          2|          Mary|2013-08-02|   57963|        ON_HOLD|       145023|                 1014|             149.94|
|          2|          Mary|2013-08-02|   57963|        ON_HOLD|       145022|                 1014|              99.96|
|          2|          Mary|2013-08-02|   57963|        ON_HOLD|       145021|                  627|             199.95|
|          2|          Mary|2013-08-02|   57963|        ON_HOLD|       145020|                 1073|             199.99|
|          2|          Mary|2013

In [ ]:
from pyspark.sql.functions import to_date

In [ ]:
from pyspark.sql import Row
from pyspark.sql.functions import sum as _sum

In [ ]:
flatten.select('customer_id','customer_fname',col("order_date"),to_date(col("order_date"),"yyyy-MM-dd").alias("order_date_converted"),'order_status','order_item_subtotal'). \
filter("order_status IN ('COMPLETE','CLOSED')"). \
groupBy(date_format('order_date_converted','yyyy-MM').alias('order_month')). \
agg(_sum('order_item_subtotal').alias('Revenue')). \
orderBy('order_month'). \
show(10)

+-----------+------------------+
|order_month|           Revenue|
+-----------+------------------+
|    2013-07|333465.45000000094|
|    2013-08| 1221828.899999967|
|    2013-09|1302255.7999999626|
|    2013-10|1171686.9199999706|
|    2013-11| 1379935.329999958|
|    2013-12|1277719.5999999633|
|    2014-01|1230221.7399999644|
|    2014-02|1217770.0899999668|
|    2014-03| 1271350.969999966|
|    2014-04|1249723.5199999677|
+-----------+------------------+
only showing top 10 rows

